In [ ]:
from GoogleNews import GoogleNews

googlenews = GoogleNews()

print(googlenews.getVersion())

googlenews.enableException(True)

googlenews = GoogleNews(period="7d")

googlenews.set_lang("en")
googlenews.set_encode("utf-8")

news_results = googlenews.get_news("APPLE")

In [ ]:
print(news_results)

In [ ]:
googlenews.total_count()

In [ ]:
import redditwarp.SYNC

client = redditwarp.SYNC.Client()

newest_comment_ts = 0

subreddits = ['bitcoin', 'cryptocurrency', 'bitcoinmarkets']
# How many subscribers does r/Python have?
subr = client.p.subreddit.fetch_by_name("bitcoinmarkets")
print(subr.subscriber_count)
hot = client.p.subreddit.pull.hot("bitcoin", amount=5)
for s in hot:
    print(s.comment_count)
    if "daily" in s.title.lower():
        print(s.title)
        print(s.created_at)
        print(s.created_ut)
        tree_node = client.p.comment_tree.fetch(s.id36, sort="new", limit=4)
        for ch in tree_node.children:
            c = ch.value
            if c.created_ut > newest_comment_ts

            print(
                f"""\
                {c.submission.id36}+{c.id36} ^{c.score}
                {c.created_ut}
                u/{c.author_display_name} says:
                {c.body}
                """
            )

In [ ]:
import feedparser
import time
import os
from pprint import pp


class RssFeed:
    def __init__(self, url):
        self.url = url
        self.latest_timestamp_file = "latest_timestamp.txt"
        self.latest_timestamp = self.get_latest_timestamp()

    def get_latest_timestamp(self):
        try:
            with open(self.latest_timestamp_file, "r") as f:
                return float(f.readline().strip())
        except (FileNotFoundError, ValueError):
            return None

    def set_latest_timestamp(self, timestamp):
        with open(self.latest_timestamp_file, "w") as f:
            f.write(str(timestamp))

    def get_article_timestamp(self, entry):
        if hasattr(entry, "published_parsed"):
            return time.mktime(entry.published_parsed)
        return time.time()

    def fetch_and_store(self):
        feed = feedparser.parse(self.url)
        new_latest_timestamp = self.latest_timestamp
        for entry in feed.entries:
            pp(entry)
            continue
            article_timestamp = self.get_article_timestamp(entry)
            if (
                self.latest_timestamp is None
                or article_timestamp > self.latest_timestamp
            ):
                # Insert the article into your storage here
                # Replace with actual insertion logic
                print(
                    f"New Article: {entry.title} - Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(article_timestamp))}"
                )
                pp(entry)
                new_latest_timestamp = (
                    max(new_latest_timestamp, article_timestamp)
                    if new_latest_timestamp
                    else article_timestamp
                )

        if new_latest_timestamp != self.latest_timestamp:
            self.set_latest_timestamp(new_latest_timestamp)
            self.latest_timestamp = new_latest_timestamp

In [ ]:
import csv

file_path = "../config/rss_feeds.dev.csv"
feeds = []
with open(file_path, newline="") as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        feeds.append(row[0])

for feed in feeds:
    print(f"Fetching {feed}")
    rss_feed = RssFeed(feed)
    rss_feed.fetch_and_store()

In [ ]:
def main():
    rss_feed = RssFeed("http://feeds.bbci.co.uk/news/rss.xml")
    while True:
        rss_feed.fetch_and_store()
        time.sleep(3600)  # Wait for 1 hour


if __name__ == "__main__":
    main()

In [ ]:
import feedparser

# Parsing the provided RSS feed URL
# feed_url = "https://www.newsbtc.com/feed/"
# feed_url = "https://nitter.cz/elonmusk/rss"
feed_url = "https://news.bitcoin.com/feed/"
feed = feedparser.parse(feed_url)

# Checking the basic information about the feed and the number of entries
feed_info = {
    "feed_title": feed.feed.get("title", "No title available"),
    "feed_subtitle": feed.feed.get("subtitle", "No subtitle available"),
    "number_of_entries": len(feed.entries),
}

feed_info